# 교차검증

In [43]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [44]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [45]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [46]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [47]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [48]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [49]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00807571, 0.00796437, 0.00809598, 0.00850582, 0.00762391]), 'score_time': array([0.00145531, 0.00121999, 0.00189114, 0.00127101, 0.00168705]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [50]:
import numpy as np
print(np.mean(scores['test_score']))

0.8554925223957948


In [51]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedGroupKFold())
print(np.mean(scores['test_score']))

NameError: name 'StratifiedGroupKFold' is not defined

# 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'min_impurity_decrease': [0.0001, 0.0002, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


In [53]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range (5,20,1),
          'min_samples_split' : range(2,100,10)}
        

In [54]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'max_depth': range(5, 20), 'min_impurity_decrease': array([0.0001... 0.0009]), 'min_samples_split': range(2, 100, 10)}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [55]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [56]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 랜덤 서치

In [ ]:
# uniform 실수 randint 정수
from scipy.stats import uniform, randint

In [58]:
rgen = randint(0,10)
rgen.rvs(10)

array([1, 5, 4, 4, 4, 6, 4, 1, 3, 3])

In [ ]:
# 중복없이 어떤 값들이 나왔는지
# 그 값들이 각각 몇번 나왔는지
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103, 102,  99, 101,  94, 101, 100,  94, 100, 106]))

In [60]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.99968716, 0.26016128, 0.67607484, 0.21501165, 0.85718175,
       0.77265648, 0.60592534, 0.60257822, 0.34600322, 0.38447796])

In [64]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25)}

In [65]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....t 0x14494d6f0>, 'min_impurity_decrease': <scipy.stats....t 0x1578b43d0>, 'min_samples_leaf': <scipy.stats....t 0x1565039d0>, 'min_samples_split': <scipy.stats....t 0x154c75c60>}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [ ]:
# 교차 검증 점수 최적값
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [68]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [69]:
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86


### TEST set 은 최종 모델 선택할 때까지 사용 X